In [1]:
import sys
sys.path.insert(0,r"C:\Users\LENOVO\Downloads")
import numpy as np
import pandas as pd
import csv
from sklearn.model_selection import train_test_split
import class_report as cr
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, auc, f1_score, log_loss
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [4]:
## Lectura de datos     
base_eje = pd.read_csv(r'C:\Users\pc\Downloads\BASE_EJE.txt', sep=r'¡', engine='python')
base_eje.head()

## Eliminación de variables: 
base_eje2 = base_eje[['CUCONUSE', 'SESUSERV', 'SERVDESC', 'SESUSUSC', 'SESUFEIN','DEPADESC', 'SESUCUSA', 'SESUSAPE','SESUCICL', 'SUSCNITC', 
        #'VECTOR','SESUCATE', 'SESUSUCA','SESULOCA','SESUSAAN','V1', 'V2', 'V3', 'V4', 'V5', 'V6',#'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 
        'TIPIFICACION_CLIENTE', 'SUMA','CANTIDAD','CALIFICACION_VECTOR_SERVICIO',
       #'VECTOR_CUALITATIVO_CONTRATO',#'BANCO_1', 'DIA_PAGO_1', 'DIA_PAGO_2','DIA_PAGO_3', 'FECHA_SUSPENSION','RANGO_EDAD', 'GENERO','PROM_SUSC', 'CANAL_ENVIO'
       'SEGMENTACION', 'REGIONAL','ESTRATO_AGRUPADO','DEPARTAMENTO_AGR','ANTIGUEDAD_DIAS', 'CANAL_PAGO']]

base_eje3 = base_eje2.dropna()

In [5]:
## Group by por nit para calificacion
b4 = base_eje3[['SUSCNITC','CALIFICACION_VECTOR_SERVICIO']]
group_nit = b4.groupby(['SUSCNITC'])['CALIFICACION_VECTOR_SERVICIO'].mean().reset_index()


## Join de la nueva calificacion
base_eje4 = pd.merge(base_eje3, group_nit, how='left', on=['SUSCNITC'])

## Prueba promedio
b5 = base_eje4[['SUSCNITC','CALIFICACION_VECTOR_SERVICIO_x','CALIFICACION_VECTOR_SERVICIO_y']]

### Creacion de la etiqueta de clasificacion
base_eje4['y'] = pd.cut(x=base_eje4['CALIFICACION_VECTOR_SERVICIO_y'],
                            bins=[-1,50,76,100],
                            labels=['No pago','Pago inoportuno','Pago'])

In [6]:
dataTypeSeries = base_eje4.dtypes
dataTypeSeries

cat = base_eje4.select_dtypes(include=['object','category'])
cuanti = base_eje4.select_dtypes(include=['int64','float64'])
  

le = LabelEncoder()
cat = cat.apply(lambda col: le.fit_transform(col))
cat.head(10)

base_eje5=cuanti.join(cat)
print(base_eje5.dtypes)

base_eje5.to_csv(r'C:\Users\pc\Desktop\base_eje_process.csv')

CUCONUSE                            int64
SESUSERV                            int64
SESUSUSC                            int64
SESUCUSA                            int64
SESUCICL                            int64
SUMA                                int64
CANTIDAD                            int64
CALIFICACION_VECTOR_SERVICIO_x      int64
ESTRATO_AGRUPADO                  float64
ANTIGUEDAD_DIAS                     int64
CALIFICACION_VECTOR_SERVICIO_y    float64
SERVDESC                            int32
SESUFEIN                            int32
DEPADESC                            int32
SESUSAPE                            int32
SUSCNITC                            int32
TIPIFICACION_CLIENTE                int32
SEGMENTACION                        int32
REGIONAL                            int32
DEPARTAMENTO_AGR                    int32
CANAL_PAGO                          int32
y                                   int32
dtype: object


In [2]:

base_eje5 = pd.read_csv(r'C:\Users\LENOVO\Downloads\base_eje_process.csv', sep=',', engine='python')


X = base_eje5[[
        #'CUCONUSE', 
        'SESUSERV', 'SERVDESC', 'SESUSUSC','DEPADESC', 'SESUCUSA', 'SESUSAPE','SESUCICL', 'TIPIFICACION_CLIENTE', 
        #'SESUFEIN','SUSCNITC', 'VECTOR','SESUCATE', 'SESUSUCA','SESULOCA','SESUSAAN','RANGO_EDAD', 'GENERO', 
        #'V1', 'V2', 'V3', 'V4', 'V5', 'V6','V7', 'V8', 'V9', 'V10', 'V11', 'V12','REGIONAL','PROM_SUSC', 'CANAL_ENVIO'     
        # 'SUMA','CANTIDAD','CALIFICACION_VECTOR_SERVICIO_x', 'CALIFICACION_VECTOR_SERVICIO_y',
        #'VECTOR_CUALITATIVO_CONTRATO','BANCO_1', 'DIA_PAGO_1', 'DIA_PAGO_2','DIA_PAGO_3', 'FECHA_SUSPENSION', 
       'SEGMENTACION','ESTRATO_AGRUPADO','DEPARTAMENTO_AGR','ANTIGUEDAD_DIAS','CANAL_PAGO']]

y = base_eje5['y']

In [3]:
from datetime import datetime
instanteInicial = datetime.now()
X_ = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.2,random_state=321)

classifiers = [
    KNeighborsClassifier(3, n_jobs = -1),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1, n_jobs = -1),
    MLPClassifier(alpha=1, max_iter=500),
    LogisticRegression(multi_class="multinomial", solver="lbfgs", C=5, n_jobs = -1),
    GaussianNB()]

# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    print("Accuracy: {:.4%}".format(acc))
    
    train_predictions = clf.predict_proba(X_test)
    ll = log_loss(y_test, train_predictions)
    print("Log Loss: {}".format(ll))
    
    log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
    log = log.append(log_entry)
    
print("="*30)

instanteFinal = datetime.now()
tiempo = instanteFinal - instanteInicial # Devuelve un objeto timedelta
segundos = tiempo.seconds

KNeighborsClassifier
****Results****
Accuracy: 94.6314%
Log Loss: 1.0568931248105895
DecisionTreeClassifier
****Results****
Accuracy: 91.4377%
Log Loss: 0.28272500713357135
RandomForestClassifier
****Results****
Accuracy: 87.8405%
Log Loss: 0.3676604697140999
MLPClassifier
****Results****
Accuracy: 89.3398%
Log Loss: 0.31413823720262685
LogisticRegression
****Results****
Accuracy: 87.6652%
Log Loss: 0.3583027605998406
GaussianNB
****Results****
Accuracy: 86.7688%
Log Loss: 1.005091098191686


In [ ]:
## El mejor modelo de clasificacion fue el KNN

In [5]:
kf = StratifiedKFold(n_splits=5,shuffle=False)

accuracy_model = []
# Iteraciones del entrenamiento sobre los kfolds
for train_index, test_index in kf.split(X, y) :
    # Particion train test para validacion cruzada
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Entrenamiento de la red MLP
    #model = MLPClassifier(alpha=1, max_iter=500).fit(X_train, y_train) 
    model = KNeighborsClassifier(3).fit(X_train, y_train) 
    accuracy_model.append(cr.class_report(y_true = y_test, 
                                       y_pred = model.predict(X_test),
                                       y_score = model.predict_proba(X_test)))
 
# Print the accuracy    
print(accuracy_model)

[             precision    recall  f1-score   support      pred       AUC
0             0.902183  0.959767  0.930084  104218.0  110870.0  0.714970
1             0.583748  0.449564  0.507944    7574.0    5833.0  0.780718
2             0.253374  0.117145  0.160216    9134.0    4223.0  0.585633
avg / total   0.833231  0.864165  0.845493  120926.0  120926.0  0.920665,              precision    recall  f1-score   support      pred       AUC
0             0.907087  0.955967  0.930886  104218.0  109834.0  0.735781
1             0.546868  0.487589  0.515530    7574.0    6753.0  0.796728
2             0.230007  0.109262  0.148148    9134.0    4339.0  0.585045
avg / total   0.833383  0.862676  0.845748  120926.0  120926.0  0.921111,              precision    recall  f1-score   support      pred       AUC
0             0.918410  0.958903  0.938220  104218.0  108813.0  0.773124
1             0.431329  0.433721  0.432521    7574.0    7616.0  0.763122
2             0.267349  0.131611  0.176389    91

In [6]:
import pickle

In [7]:
# save

filename='mdl_eje.sav'

pickle.dump(model,open(filename,'wb'))